In this exercise, you'll build a project management assistant using OpenAI API Function Calling
A .csv file is used to simulate reading and writing from a database or project management tool API.  Follow the directions in the starter code below, and try to build the functions and function calling logic before you look at the solution on the next page! 

Import necessary libraries

In [1]:
import os, openai
openai.api_base = "https://openai.vocareum.com/v1"
import json
import pandas as pd

# OpenAI key here.
openai.api_key = os.environ["OPENAI_API_KEY"]

First, define the Python functions that will read and write from the project_management.csv file using Pandas dataframes. This code uses Pandas dataframes to read and write from the .csv file. 

We define 3 tasks our project management assistant can perform. 

Each function returns a JSON string as output

In [14]:
# Load the project management data
df = pd.read_csv('project_management.csv')
print(df)

def task_retrieval_and_status_updates(task_id, status, last_updated):
    """Retrieve and update task status"""
    df.loc[df['Task ID'] == task_id, 'Status'] = status
    df.loc[df['Task ID'] == task_id, 'Last Updated'] = last_updated
    df.to_csv('project_management.csv', index=False)  # save changes to file
    task = df.loc[df['Task ID'] == task_id]
    print(f'task_retrieval_and_status_updates: {json.dumps(task.to_dict())}')
    print("*" * 50)
    return json.dumps(task.to_dict())

def project_reporting_and_analytics(project_id):
    """Generate reports on project progress and team performance"""
    project = df.loc[df['Project ID'] == project_id]
    print(f'project_reporting_and_analytics: {json.dumps(project.to_dict())}')
    print("*" * 50)
    return json.dumps(project.to_dict())

def resource_allocation_and_scheduling(task_id, assigned_to, time_estimate, due_date, status):
    """Allocate tasks based on current workloads and schedules"""
    df.loc[df['Task ID'] == task_id, 'Assigned To'] = assigned_to
    df.loc[df['Task ID'] == task_id, 'Time Estimate'] = time_estimate
    df.loc[df['Task ID'] == task_id, 'Due Date'] = due_date
    df.loc[df['Task ID'] == task_id, 'Status'] = status
    df.to_csv('project_management.csv', index=False)  # save changes to file
    # Christoph: added one line:
    task = df.loc[df['Task ID'] == task_id]
    print(f'resource_allocation_and_scheduling: {json.dumps(task.to_dict())}')
    print("*" * 50)
    return json.dumps(task.to_dict())

   Task ID               Task Name  Project ID    Assigned To       Status  \
0        1  Design Database Schema         101       Jane Doe  In Progress   
1        2    Implement Login Page         101     John Smith    completed   
2        3  Prepare Project Report         102  Alice Johnson    Completed   

  Priority    Due Date Date Created Last Updated  Time Estimate  Time Spent  \
0     High  2023-08-01   2023-07-01   2023-07-10             10         4.0   
1   Medium  2023-08-15   2023-07-01   2022-05-20              5         NaN   
2      Low  2023-07-15   2023-06-01   2023-07-05              2         2.0   

                                        Description   Project Phase  \
0  Create initial database schema for customer data          Design   
1           "Develop the login page UI and backend"  Implementation   
2          Compile the weekly project status report       Reporting   

   Dependencies  
0           NaN  
1           1.0  
2           NaN  


Next, we'll build the project management assistant conversation. 

We'll define the messages to send to the model, including a tools dictionary that defines a list of tools, which are the functions that are available to the model to identify and parse parameters for. 

In [17]:
def run_conversation():
    messages = [
        {"role": "system", "content": "You are a project management assistant with knowledge of project statuses, task assignments, and scheduling. You can provide updates on projects, assign tasks to team members, and schedule meetings. You understand project management terminology and are capable of parsing detailed project data. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
 },
        {"role": "user", "content": "Change the status of task 2 to completed."} # this prompt should call task_retrieval_and_status_updates
    ]
    tools= [
        {
            "type": "function",
            "function": {
                "name": "task_retrieval_and_status_updates",
                "description": "Retrieve and update task status",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "task_id": {
                            "type": "integer",
                            "description": "The unique identifier for the task"
                        },
                        "status": {
                            "type": "string",
                            "description": "The new status of the task"
                        },
                        "last_updated": {
                            "type": "string",
                            "description": "The date of the last status update or change to the task"
                        }
                    },
                    "required": ["task_id", "status", "last_updated"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "project_reporting_and_analytics",
                "description": "Generate reports on project progress and team performance",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "project_id": {
                            "type": "integer",
                            "description": "The unique identifier for the project"
                        }
                    },
                    "required": ["project_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "resource_allocation_and_scheduling",
                "description": "Allocate tasks based on current workloads and schedules",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "task_id": {
                            "type": "integer",
                            "description": "The unique identifier for the task"
                        },
                        "assigned_to": {
                            "type": "string",
                            "description": "The user ID or name of the person to whom the task is assigned"
                        },
                        "time_estimate": {
                            "type": "integer",
                            "description": "An estimate of the time required to complete the task"
                        },
                        "due_date": {
                            "type": "string",
                            "description": "The deadline for the task completion"
                        },
                        "status": {
                            "type": "string",
                            "description": "The current status of the task"
                        }
                    },
                    "required": ["task_id", "assigned_to", "time_estimate", "due_date", "status"]
                }
            }
        }
    ]
    # Use openai.ChatCompletion.create for openai < 1.0
    # openai.chat.completions.create for openai > 1.0
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # let the model decide which tool (function) to use
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls # get the tool calls from the first response
    print(tool_calls)
    # end of first response, now we parse the response and call the functions the model identified from our tool list
    # check if the model wanted to call a function
    if tool_calls:
        # list the available functions and their corresponding python functions
        available_functions = {
            "task_retrieval_and_status_updates": task_retrieval_and_status_updates,
            "project_reporting_and_analytics": project_reporting_and_analytics,
            "resource_allocation_and_scheduling": resource_allocation_and_scheduling,
        }  
        messages.append(response_message)  # extend the conversation with the first response
        # send the info for each function call and function response to the model
        for tool_call in tool_calls: # iterate through the tool calls in the response
            function_name = tool_call.function.name # get the name of the function to call
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = "Error: no function call done."
            if function_name == 'task_retrieval_and_status_updates': # call the Python function
                function_response = function_to_call(
                    task_id=function_args.get("task_id"),
                    status=function_args.get("status"),
                    last_updated=function_args.get("last_updated")
                )
            elif function_name == 'project_reporting_and_analytics':
                # Fill in the function call for the project_reporting_and_analytics function with parameters from the function_args dictionary
                function_response = function_to_call(
                    project_id=function_args.get("project_id"),
                )
            elif function_name == 'resource_allocation_and_scheduling':
                # Fill in the function call for the resource_allocation_and_scheduling function with parameters from the function_args dictionary
                function_response = function_to_call(
                    task_id = function_args.get("task_id"),
                    assigned_to = function_args.get("assigned_to"),
                    time_estimate = function_args.get("time_estimate"),
                    due_date = function_args.get("due_date"),
                    status = function_args.get("status"),
                )

            message_to_append = {
                    "tool_call_id": tool_call.id, #
                    "role": "tool",
                    "name": function_name,
                    "content": function_response, # send the function response to the model, it's the JSON string of the function response
                }
            messages.append(message_to_append)  # extend conversation with function response

        # Christoph:
        print(f"# messages: {len(messages)}.")
        for msg in messages:
            print(f"Message: {msg}.")
            print("=" * 50)
        print()
        print("=" * 50)
        # See https://gist.github.com/gaborcselle/2dc076eae23bd219ff707b954c890cd7
        #messages[1].content = "" # clear the first message (parsing bug)
        messages[1]['content'] = "" # clear the first message (parsing bug)

        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response

In [18]:
print("*" * 50)
print(run_conversation()) # will print the second response from the model

**************************************************
[<OpenAIObject id=call_BaWA2TcL10FUKW29VCWBLsBd at 0x2284d2c1860> JSON: {
  "function": {
    "arguments": "{\"task_id\":2,\"status\":\"completed\",\"last_updated\":\"2023-07-15\"}",
    "name": "task_retrieval_and_status_updates"
  },
  "id": "call_BaWA2TcL10FUKW29VCWBLsBd",
  "type": "function"
}]
task_retrieval_and_status_updates: {"Task ID": {"1": 2}, "Task Name": {"1": "Implement Login Page"}, "Project ID": {"1": 101}, "Assigned To": {"1": "John Smith"}, "Status": {"1": "completed"}, "Priority": {"1": "Medium"}, "Due Date": {"1": "2023-08-15"}, "Date Created": {"1": "2023-07-01"}, "Last Updated": {"1": "2023-07-15"}, "Time Estimate": {"1": 5}, "Time Spent": {"1": NaN}, "Description": {"1": " \"Develop the login page UI and backend\""}, "Project Phase": {"1": "Implementation"}, "Dependencies": {"1": 1.0}}
**************************************************
# messages: 4.
Message: {'role': 'system', 'content': "You are a project mana